In [ ]:
def wrangle(df):
    #removing features with > 50% of observatins missing
    observation_threshold = len(df)/2
    df = df.dropna(thresh=observation_threshold , axis = 1)
    #removing redundant/unnecessary columns 
    df.drop(columns='converted_pledged_amount', inplace = True) #redundant with usd_pledged
    df.drop(columns='creator', inplace = True) #high cardinality/not beneficial for class seperability
    df.drop(columns='currency', inplace=True)#will use usd pledged for all
    df.drop(columns='currency_symbol', inplace=True)
    df.drop(columns='currency_trailing_code', inplace=True)
    df.drop(columns='current_currency', inplace = True)
    df.drop(columns='fx_rate', inplace=True) #redundant with static_usd_rate
    df.drop(columns='photo', inplace=True) #will not be using photos for prediction
    df.drop(columns='pledged', inplace=True) #Has different currency values/not standardized
    df.drop(columns='profile', inplace=True) #user info 
    df.drop(columns='slug', inplace=True) #high cardinality/redundant with name column
    df.drop(columns='source_url', inplace=True) #url of each project
    df.drop(columns='state_changed_at', inplace=True) #high correlation with deadline
    df.drop(columns='urls', inplace=True) #more url data
    df.drop(columns='usd_type', inplace=True) #redundant as all campaigns will be in usd
    
    # Target Leakage
    df.drop(columns='spotlight', inplace=True) #target leakage--leaks info about project success beforehand
    df.drop(columns='staff_pick', inplace=True) #we don't know if a project will be a staff pick
    df.drop(columns='usd_pledged', inplace=True) #nothing has been pledged yet 
    df.drop(columns='backers_count', inplace=True) # leaks info about future, we do not know how many backers a campaign will have
    
    #remove observations with canceled/live as target class
    class_to_drop1 = df[df['state'] == 'canceled'].index
    class_to_drop2 = df[df['state'] == 'live'].index
    df.drop(class_to_drop1, inplace=True)
    df.drop(class_to_drop2, inplace=True)
    
    #remove null observations for blurb feature, location
    df.dropna(subset=['blurb'], inplace=True)
    df.dropna(subset=['location'], inplace=True)
    
    #remove columns with only 1 unique value/won't aid in class seperability
    df.drop(columns='is_starrable', inplace=True)
    df.drop(columns='disable_communication', inplace=True)
    
    #drop duplicated campaigns
    df.drop_duplicates('id',inplace=True)
    
    #set id column as index 
    df.set_index('id',inplace=True)
    df.reset_index(drop=True,inplace=True)
    
    #convert unix time features to datetime object
    cols = ['created_at', 'deadline', 'launched_at']
    for col in cols:
        df[col] = pd.to_datetime(df[col], origin='unix', unit='s')
        
    ##goal feature needs to be standardized to usd. It is in local currency of campaign country
    #new feature for goal in usd
    df['goal_usd'] = round(df['goal'] * df['static_usd_rate'],2) 
    #drop goal feature with different currencies & conversion rate feature
    df.drop(columns='goal', inplace=True)
    df.drop(columns='static_usd_rate',inplace=True)
    
    #remove location and country_displayable_name features/redundant with country
    df.drop(columns='location', inplace=True)
    df.drop(columns='country_displayable_name', inplace=True)
    
    #dropping name column due to high cardinality 
    df.drop(columns='name', inplace=True)
    
    #creating new feature for campaign duration
    df['campaign_duration_in_days'] = df['deadline'] - df['launched_at']
    df['campaign_duration_in_days'] = df['campaign_duration_in_days'].dt.round('d').dt.days
    
    
    
    
    return df